## Basic Operations on Encrypted Data

This tutorial is located in the `tutorial/01_heseries_basic_operation.ipynb` path of the HEaaN.Stat Docker image.

### Step 1. Import HEaaN.Stat SDK    
Along with HEaaN.Stat, you will also need libraries such as `numpy` and `pandas` for handling plaintext data processing.

In [11]:
import os
os.environ["HEAAN_TYPE"] = "pi"
os.environ["OMP_NUM_THREADS"] = "8"

import numpy as np
import pandas as pd

import heaan_stat
from heaan_stat import Context, HESeries


### Step 2. Create a Context
A `Context` holds necessary internal information for performing homomorphic encryption, including HE encryption keys.

HE keys used for encryption and decryption are managed by the `Keypack` class. When a ciphertext is encrypted using a particular `Keypack`, any subsequent operations on that ciphertext must use the same `Keypack`; otherwise, the results will be invalid.

Since the `Context` contains all the essential elements needed to interact with the HEaaN CKKS library, nearly every object in the HEaaN.Stat requires a Context to function properly.

The `Context` also handles configuration settings, including whether to generate new keys for encryption and decryption, and which types of keys should be loaded. 



In [1]:
from heaan_stat import Context

context = Context(
    key_dir_path='./keys',
    generate_keys=True,  # To use existing keys, set it to False or omit this
)

context # Show the values of parameters in the Context

/tmp/ipykernel_3379/2287943210.py:3: UserWarning: The currently available memory size is 3.8GB. It must be at least 10.2GB for `FGb` parameter preset to compute operations.
  context = Context(


Context(
  preset: FGb,
  key dir path: "keys",
  loaded key type: "all",
  GPU: Not available
)

### Step 3. HESeries creation

Creating an `HESeries`, an extension of Pandas Series with homomorphic encryption (HE), by passing a list of values or a Pandas Series directly.

In [2]:
from heaan_stat import HESeries

hs1 = HESeries(context, [1,2,3,4,5])
hs1 # Show information of HESeries hs1

HESeries(
  encrypted: False,
  length: 5,
  dtype: numeric,


To convert the `HESeries` to a Pandas `Series`, use the `to_series()` or `decode()` function. However, if the `HESeries` is encrypted, attempting to convert it directly will raise an error. Instead, use `decrypt_decode()` to decrypt and convert the data, or `decrypt(inplace=False).decode()` to keep the original data encrypted while obtaining the decrypted version as a Pandas `Series`.

In [3]:
series1 = hs1.to_series()
type(series1), series1 # Show the type and values of series1

(pandas.core.series.Series,
 0    1
 1    2
 2    3
 3    4
 4    5
 dtype: int64)

You can also initialize the `HESeries` in encrypted state by setting the encrypt parameter to `True`.

In [4]:
hs2 = HESeries(context, [1,3,5,7,9], encrypt=True)
hs2 # Show information of HESeries hs2

HESeries(
  encrypted: True,
  level: 12,
  length: 5,
  dtype: numeric,

To convert the encrypted `HESeries` to plaintext, decrypt it first using the `decrypt()` function, then convert it to a Pandas `Series`.


In [5]:
series = hs2.decrypt(False).to_series()
series, hs2 # Show the pandas Series, and encrypted HESeries

(0    1
 1    3
 2    5
 3    7
 4    9
 dtype: int64,
 HESeries(
   encrypted: True,
   level: 12,
   length: 5,
   dtype: numeric,)

`HESeries` supports various data types, including numeric, boolean, and categorical.

In [12]:
category = pd.Series(np.random.randint(1, 4, size=5), dtype="int")
hs3 = HESeries(context, category)
hs3
# Show information of HESeries hs3
series3 = hs3.to_series()
type(series3), series3 # Show the type and values of series3
# Show the type and values of series3

(pandas.core.series.Series,
 0    2
 1    3
 2    2
 3    1
 4    1
 dtype: int64)

In [13]:
hs3.to_series()

0    2
1    3
2    2
3    1
4    1
dtype: int64

### Step 4. Basic Operation

You can perform arithmetic operations such as addition (`+`), subtraction (`-`), and multiplication (`*`) directly on `HESeries` objects. These operations work regardless of whether the data is in plaintext or ciphertext.

If at least one of the operands is an encrypted `HESeries`, the result of the operation will also be encrypted. This ensures that sensitive data remains secure throughout computations.

Let’s start by creating two encrypted `HESeries` objects:



In [14]:
col1 = HESeries(context, [1,2,3,4], encrypt=True)
col2 = HESeries(context, [3,-2,0,1], encrypt=True)
# Generate 2 HESeries object with encrypt option 'True'

#### Addition
You can add the two `HESeries` as follows:

In [15]:
add_col = col1 + col2
print("ADD")
add_col.decrypt().to_series()

ADD


0    4
1    0
2    3
3    5
dtype: int64

#### Subtraction
Similarly, subtraction can be performed:

In [16]:
sub_col = col1 - col2
print("SUB")
sub_col.decrypt().to_series()

SUB


0   -2
1    4
2    3
3    3
dtype: int64

#### Multiplication
You can also multiply the two `HESeries`:

In [1]:
from heaan_stat import Context, HESeries

context = Context(
    key_dir_path="./keys",
    generate_keys=False
)

col1 = HESeries(context, [1.0, 2.0, 3.0])
col2 = HESeries(context, [4.0, 5.0, 6.0])
mult_col = col1 * col2

print("MULT")

# ✅ 복호화되어 있는지 확인 후 처리
if mult_col.encrypted:
    mult_col.decrypt()

print(mult_col.to_series())


/tmp/ipykernel_14084/3011891095.py:3: UserWarning: The currently available memory size is 5.6GB. It must be at least 10.2GB for `FGb` parameter preset to compute operations.
  context = Context(


MULT
0     4.0
1    10.0
2    18.0
dtype: float64


In [ ]:
category = pd.Series(np.random.randint(1, 4, size=5), dtype="int")
hs3 = HESeries(context, category)
hs3
# Show information of HESeries hs3
series3 = hs3.to_series()
type(series3), series3 # Show the type and values of series3
# Show the type and values of series3

(pandas.core.series.Series,
 0    2
 1    3
 2    2
 3    1
 4    1
 dtype: int64)